# Understand the problem

**House Prices - Advanced Regression Techniques**

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

[Kaggle page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)

# Setup

In [ ]:
import os
# Where to save the figures
CURRENT_DIR = "." 
#The dot . is the current directory; 
# .. refers to one directory further up in the hierarchy
IMAGES_PATH = os.path.join(CURRENT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
import sklearn
from sklearn.preprocessing import StandardScaler
from scipy import stats
%matplotlib inline
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import pandas as pd

# EDA

### Get an overall view of the data

In [ ]:
#read data


In [ ]:
#drop column ID:


In [ ]:
#check columns


In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.describe()

#### Check missing data and drop columns with high missing rate

In [ ]:
#check missing data
def missing_values_table(df):
        #1 Total missing values
        mis_val = df.isnull().sum()
        
        #2 Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        #3 Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        #4 Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        #5 Only keep the columns with missing values
        mis_val_table_only = mis_val_table_ren_columns.loc[mis_val_table_ren_columns['% of Total Values'] > 0]
        
        #6 Return the dataframe with missing information
        return mis_val_table_only

In [ ]:
missing_values_table(df_train)

*Practice:*
Let's break down this function into small pieces and run each step
one by one to better understand it

In [ ]:
#step one: Total missing values


In [ ]:
#step two: Percentage of missing values


In [ ]:
#step three: Make a table with the results
     #pd.concat - Concatenate pandas objects along a particular axis (1 is by column)


In [ ]:
#Step four: Rename the columns


In [ ]:
#Step Five: Only keep the columns with missing rate > 0


In [ ]:
#Step Six: Apply the function to our dataframe:


Usually there are three options to deal with missing values:
 1. Imputation
 2. Create missing flag
 3. Drop columns with a high percentage of missing vlaues

We see there are a number of columns with a high percentage of missing values. 
There is no well-established threshold for removing missing values, 

and the best course of action depends on the problem. 

Here, to reduce the number of features, we will remove any columns that have greater than 30% missing rate (in real situations, the threshold can be 90%).

In [ ]:
## find columns with missing > 30%


In [ ]:
# drop these columns


In [ ]:
# reapply this missing function


#### Understand the target variable - SalePrice

In [ ]:
#histogram


In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

**Skewness** is a measure of asymmetry of a distribution.
    
- When the value of the skewness is negative, the tail of the distribution is longer towards the left hand side of the curve
    
- When the value of the skewness is positive, the tail of the distribution is longer towards the right hand side of the curve

*Important Notes*:

- If the skewness is between -0.5 and 0.5, the data are fairly symmetrical

- If the skewness is between -1 and — 0.5 or between 0.5 and 1, the data are moderately skewed

- If the skewness is less than -1 or greater than 1, the data are highly skewed  


**Kutosis** determine the volume of the outlier
- If the distribution is tall and thin it is called a leptokurtic distribution(Kurtosis > 3)  
- A flat distribution where the values are moderately spread out (i.e., unlike leptokurtic) is called platykurtic(Kurtosis <3) distribution
- A distribution whose shape is in between a leptokurtic distribution and a platykurtic distribution is called a mesokurtic(Kurtosis=3) distribution. A mesokurtic distribution looks more close to a normal distribution



##### Relationship with numerical variables:

In [ ]:
#scatter plot GrLivArea vs. SalePrice
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
#scatter plot TotalBsmtSF vs. SalePrice


##### Relationship with categorical variables:

In [ ]:
#box plot OverallQual vs. SalePrice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

In [ ]:
#box plot YearBuilt SalePrice

### Explore all the features and gain insights

- Numerical features

In [ ]:
# Finding numeric features


In [ ]:
#plot all numerical features
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8); 
save_fig("num_histogram_plots")

Features such as `1stFlrSF`, `TotalBsmtSF`, `LotFrontage`, `GrLiveArea` have a similar distribution to `SalePrice`'s

In [ ]:
#find which features are strongly correlated with SalePrice and store them into a variable - top_corr_features
df_num_corr = df_num.corr()['SalePrice'][:-1] # -1 because the latest row is SalePrice
top_corr_features = df_num_corr[abs(df_num_corr) > 0.5].sort_values(ascending=False)
print("There is {} strongly correlated values with SalePrice:\n{}".format(len(top_corr_features), top_corr_features))

In [ ]:
#use heatmap to see if some variables are linked between each other 
corr = df_num.drop('SalePrice', axis=1).corr() # We already examined SalePrice correlations
plt.figure(figsize=(12, 10))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

A lot of features seems to be correlated between each other but some of them such as YearBuild/GarageYrBlt may just indicate a price inflation over the years. As for 1stFlrSF/TotalBsmtSF, it is normal that the more the 1st floor is large (considering many houses have only 1 floor), the more the total basement will be large.

We can conclude that, by essence, some of those features may be combined between each other in order to reduce the number of features (`1stFlrSF`/`TotalBsmtSF`, `GarageCars`/`GarageArea`)

Some of the features of our dataset looks like numerical but are categorical. To separate the categorical from quantitative features lets refer ourselves to the data_description.txt file. According to this file we end up with the folowing columns:

In [ ]:
quantitative_features_list = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']
df_quantitative_values = df_num[quantitative_features_list]
df_quantitative_values.head()

Still, we have a lot of features to analyse here so let's take the strongly correlated quantitative features from this dataset and analyse them one by one

In [ ]:
features_to_analyse = [x for x in quantitative_features_list if x in top_corr_features]
features_to_analyse.append('SalePrice')
features_to_analyse

In [ ]:
fig, ax = plt.subplots(round(len(features_to_analyse) / 3), 3, figsize = (18, 12))

for i, ax in enumerate(fig.axes):
    if i < len(features_to_analyse) - 1:
        sns.regplot(x=features_to_analyse[i],y='SalePrice', data=df_train[features_to_analyse], ax=ax)

- Categorical features

Lets get all the categorical features of our dataset and see if we can find some insight in them. Instead of opening back our data_description.txt file and checking which data are categorical, lets just remove quantitative_features_list from our entire dataframe

In [ ]:
# quantitative_features_list[:-1] as the last column is SalePrice and we want to keep it
categorical_features = [a for a in quantitative_features_list[:-1] + df_train.columns.tolist() if (a not in quantitative_features_list[:-1]) or (a not in df_train.columns.tolist())]
df_categ = df_train[categorical_features]
df_categ.head()

we only care about non-numerical features now

In [ ]:
df_not_num = df_categ.select_dtypes(include = ['O'])
print('There is {} non numerical features including:\n{}'.format(len(df_not_num.columns), 
                                                                 df_not_num.columns.tolist()))

In [ ]:
fig, axes = plt.subplots(round(len(df_not_num.columns) / 3), 3, figsize=(12, 30))

for i, ax in enumerate(fig.axes):
    if i < len(df_not_num.columns):
        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=45)
        sns.countplot(x=df_not_num.columns[i], alpha=0.7, data=df_not_num, ax=ax)

fig.tight_layout()

We can see that some categories are predominant for some features such as `Utilities`, `Heating`, `GarageCond`, `Functional`... These features may not be relevant for our predictive model

In [ ]:
for i in list(df_not_num.columns):
    data = pd.concat([df_train['SalePrice'], df_train[i]], axis=1)
    f, ax = plt.subplots(figsize=(16, 8))
    fig = sns.boxplot(x=i, y="SalePrice", data=data)
    fig.axis(ymin=0, ymax=800000);
    plt.xticks(rotation=90);

### Model Preparation